In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample=pd.read_csv(r'/kaggle/input/sf-crime/sampleSubmission.csv.zip')
sample.head()

In [ ]:
sample.shape

In [ ]:
train=pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:

train.info()

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.head()

In [ ]:
train = pd.concat([train, pd.get_dummies(train['DayOfWeek'])], axis=1)
train.drop('DayOfWeek', axis=1, inplace=True)

test = pd.concat([test, pd.get_dummies(test['DayOfWeek'])], axis=1)
test.drop('DayOfWeek', axis=1, inplace=True)

In [ ]:
train = pd.concat([train, pd.get_dummies(train['PdDistrict'])], axis=1)
train.drop('PdDistrict', axis=1, inplace=True)

test = pd.concat([test, pd.get_dummies(test['PdDistrict'])], axis=1)
test.drop('PdDistrict', axis=1, inplace=True)

In [ ]:
train.drop(['Address', 'Descript', 'Resolution', 'Dates'], axis=1, inplace=True)

test.drop(['Address', 'Dates'], axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(train.shape,test.shape)

In [ ]:
X=(train.drop(['Category'],axis=1))
y=train['Category']

In [ ]:
Y=pd.get_dummies(y)
Y.head()

In [ ]:
print(X.shape,Y.shape)

In [ ]:
X.shape[1]

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.25)
print(Xtrain.shape,ytrain.shape)
print(Xtest.shape,ytest.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation

In [ ]:
model=Sequential()

model.add(Dense(256,input_shape=(X.shape[1],)))
model.add(Activation('relu'))

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(39))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.fit(Xtrain,ytrain,
         batch_size=256,
         epochs=10,
         verbose=2,
         validation_data=(Xtest,ytest))

In [ ]:
pred=model.predict(test.drop('Id',axis=1))
pred

In [ ]:
pred_final=[np.argmax(i) for i in pred]

In [ ]:
predreshape = np.max(pred, axis=1).reshape(-1, 1)
pred_final = np.array((pred == predreshape), dtype='int32')
pred_final

In [ ]:
column=list(sample.columns)
column.remove('Id')
print(column)

In [ ]:
Submission=pd.DataFrame({'Id':test['Id']})
for i,v in enumerate(column):
    Submission[v]=pred_final[:,i]
    
Submission.head()

In [ ]:
Submission.shape

In [ ]:
Submission.to_csv('Submission.csv', index=False)